In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

from superRes.fid_loss import FIDLoss
from superRes.generators import *
import geffnet
import torchvision

In [5]:
path = Path('./dataset/')

path_fullRes = path/'DIV2K_train_HR'
path_lowRes = path/'DIV2K_train_LR_mild'

In [6]:
def det_DIV2k_data(bs:int, sz:int):
    lowResSuffix = 'x4m'
    src = ImageImageList.from_folder(path_lowRes).split_by_idxs(train_idx=list(range(0,800)), valid_idx=list(range(800,900)))

    data = (src.label_from_func(lambda x: path_fullRes/(x.name).replace(lowResSuffix, '')).transform(
            get_transforms(
                max_zoom=2.
            ),
            size=sz,
            tfm_y=True,
        ).databunch(bs=bs, num_workers=8, no_check=True).normalize(imagenet_stats, do_y=True))
    data.c = 3
    return data

In [7]:
data = det_DIV2k_data(bs=10, sz=256)
# data.show_batch()
x, y = data.one_batch()

In [ ]:
# calculate_fid(x, y)

In [9]:
bs=25
sz=128
lr = 1e-3
wd = 1e-3
keep_pct=1.0
epochs = 10
nf_factor = 2

model = geffnet.mobilenetv3_100

learn_gen = gen_learner_wide(data=data,
                             gen_loss=calculate_fid,
                             arch = model,
                             nf_factor=nf_factor)

In [12]:
learn_gen.fit_one_cycle(1, lr)

epoch,train_loss,valid_loss,time


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn